# Evaluation <a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #plots>Plots</a></li>
</ol>

## Imports

In [ ]:
import os
import shutil

In [ ]:
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array

import ntpath

import copy
import re

## Hilfsfunktionen

### Enum für Trainingsset

In [ ]:
class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Enum für Label-Typ

In [ ]:
class LabelType(Enum):
    ANGULAR = 1
    STEREOGRAPHIC = 2

### Trainingsset-Typ nach String Converter

In [ ]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### LabelType nach String

In [ ]:
def labeltype_to_string(lt):
    if lt == LabelType.ANGULAR:
        return 'Angular'
    if lt == LabelType.STEREOGRAPHIC:
        return 'Stereographic'
    else:
        print('Unknown LabelType')
        return None

#### Required format of parameters parameter for _model_predict_ (...)

In [ ]:
p = {
    'dataset_to_use':[],
    'model_to_load':[],
    'dataset_name':'combined_dataset',
    'activation_function':[],
    'leaky_ReLU_alpha':[],
    'first_neuron':[],
    'dropout_rate':[],
    'hidden_layers':[],
    'optimizer':[],
    'learning_rate':[],
    'loss_function':[],
    'label_type':[]
}

## Hilfsfunktionen

### Konvertierung ($S_x$, $S_y$) $\rightarrow$ ($\phi$, $\theta$)

In [ ]:
def convert_from_stereographic(sx, sy, r = 1):
    
    l = np.sqrt(sx * sx + sy * sy)
    theta = 90 - 2 * np.degrees(np.arctan(l / 2))
    
    if sx < 0:
        phi = 180 - np.degrees(np.arcsin(sy / l))
        
    elif sx >= 0:
        if sy > 0:
            phi = np.degrees(np.arcsin(sy / l))
        elif sy < 0:
            phi = 360 + np.degrees(np.arcsin(sy / l))
        else:
            #phi1 = np.NaN
            phi = 0
    else:
        print('sx and sy undefined. should not have reached here')

    return phi, theta

### Konvertierung ($\phi$, $\theta$) $\rightarrow$ ($S_x$, $S_y$)

In [ ]:
def convert_from_spheric(phi, theta, r = 1):
    m = 2 * r * np.tan(np.radians((90 - theta) / 2))
    sy = m * np.sin(np.radians(phi))
    sx = m * np.cos(np.radians(phi))
    return sx, sy

### Radians $\rightarrow$ Degree

In [ ]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [ ]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [ ]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [ ]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [ ]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [ ]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [ ]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [ ]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

## Normierte sphärische Koordinaten

### Normierte $\rightarrow$ Sphärische

In [ ]:
def normalized_to_spheric(n_phi, n_theta):
    phi = n_phi * 180 + 180
    theta = n_theta * 45 + 45
    
    return phi, theta

# Evaluation <a name = "setup"></a>
<p><a href = #Top>Up</a>
<p><a href = #plots>Plots</a>

In [ ]:
run = 'SYNTH'
loss = 'MSE'
dataset_name = '2020-05-28'
net_index = [99, 204, 195]
APPENDIX = ['Angular', 'Normalized', 'Stereographic']
_note = ['', '', '_Custom-MAE']

eval_dir = '..\\output\\{}_Regression_{}\\Graphical_Evaluation\\'.format(run, loss)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

net_results = [None, None, None]

for i in range(3):
    with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index[i], APPENDIX[i], _note[i]), "rb") as fp:   # Unpickling
        net_results[i] = pickle.load(fp)

In [ ]:
for i in range(3):
    print(net_results[i])
    print()
    print()
    print('---------------------------------------------------------------------')
    print()
    print()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.text as text
from matplotlib.pyplot import figure

from matplotlib import cm

import tikzplotlib
%matplotlib inline

Enable_Plotting = True

# Plots <a name = "plots">
<p></a><a href = #Top>Up</a>
<p><a href = #setup>Setup</a>

In [ ]:
if Enable_Plotting:
    #interesting_cases = [0, 3, 4, 5]
    interesting_cases = [0, 5]
    interesting_cases = [0, 5]
    
    net_len = len(net_results)
    net_len = 3
    
    net_df = [[], [], []]
    
    for net_idx in range(net_len):
        net = net_results[net_idx]
        for run_idx in interesting_cases:
            df = net[run_idx][1]
            net_df[net_idx].append(df)
            #print(df)
    print(net_df[2][0])

In [ ]:
print(net_df[0][0])

##  Elevation-Dependent Angular Error

In [ ]:
def fast_elev_dependent_angular_synthetic(dataframe, eval_path, eval_file, elevation_normalized = False, store_to_file = False):
    elev_depenent_angular_error =  pd.DataFrame(data=None, index=None, columns=['elevation_angle', 'angular_error'])
    
    operation_df = dataframe
    
    dictionary = {}
    
    for index, row in operation_df.iterrows():
        true_elevation = row['Elevation_true'] * 45.0 + 45.0 if elevation_normalized else row['Elevation_true']
        key = round(true_elevation, 1)
        if key not in dictionary.keys():
            e = row['dot_angular_err_elevation']
            a = 1
            dictionary[key] = [e, a]
            
        else:
            e = dictionary[key][0]
            a = dictionary[key][1]
            e = e + row['dot_angular_err_elevation']
            a = a + 1
            dictionary[key] = [e, a]
            
    for key, value in dictionary.items():
        avg_error = value[0] / value[1]
        elev_depenent_angular_error = elev_depenent_angular_error.append(pd.DataFrame(
            data = [[key, avg_error, value[1]]], 
            index = None, 
            columns = ['elevation_angle', 'angular_error', 'No Images']))

    elev_depenent_angular_error = elev_depenent_angular_error.sort_values(by=['elevation_angle'])

    #cmse_real_df_error_azimuth.to_csv(evaluation_path + 'Net_568_Real_TD_Azimut_Fehler_in_Abhängigkeit_der_Elevation.csv', index = False)
    if(store_to_file):
        elev_depenent_angular_error.to_csv(eval_path + eval_file, index = False)
    return elev_depenent_angular_error




def fast_elev_dependent_azimuth_synthetic(dataframe, eval_path, eval_file, elevation_normalized = False, store_to_file = False):
    elev_depenent_angular_error =  pd.DataFrame(data=None, index=None, columns=['elevation_angle', 'azimuth_error'])
    
    operation_df = dataframe
    
    dictionary = {}
    
    for index, row in operation_df.iterrows():
        true_elevation = row['Elevation_true'] * 45.0 + 45.0 if elevation_normalized else row['Elevation_true']
        key = round(true_elevation, 1)
        if key not in dictionary.keys():
            e = row['Azimuth_err']
            a = 1
            dictionary[key] = [e, a]
            
        else:
            e = dictionary[key][0]
            a = dictionary[key][1]
            e = e + row['Azimuth_err']
            a = a + 1
            dictionary[key] = [e, a]
            
    for key, value in dictionary.items():
        avg_error = value[0] / value[1]
        elev_depenent_angular_error = elev_depenent_angular_error.append(pd.DataFrame(
            data = [[key, avg_error, value[1]]], 
            index = None, 
            columns = ['elevation_angle', 'azimuth_error', 'No Images']))

    elev_depenent_angular_error = elev_depenent_angular_error.sort_values(by=['elevation_angle'])

    #cmse_real_df_error_azimuth.to_csv(evaluation_path + 'Net_568_Real_TD_Azimut_Fehler_in_Abhängigkeit_der_Elevation.csv', index = False)
    if(store_to_file):
        elev_depenent_angular_error.to_csv(eval_path + eval_file, index = False)
    return elev_depenent_angular_error




def fast_elev_dependent_stereographic_synthetic(dataframe, eval_path, eval_file, elevation_normalized = False, store_to_file = False):
    elev_depenent_stereographic_error =  pd.DataFrame(data=None, index=None, columns=['elevation_angle', 's_x_error', 's_y_error'])
    
    operation_df = dataframe
    
    dictionary = {}
    
    for index, row in operation_df.iterrows():
        true_elevation = row['Elevation_true'] * 45.0 + 45.0 if elevation_normalized else row['Elevation_true']
        key = round(true_elevation, 0)
        if key not in dictionary.keys():
            e_x = row['S_x_err']
            e_y = row['S_y_err']
            a = 1
            dictionary[key] = [e_x, e_y, a]
            
        else:
            e_x = dictionary[key][0]
            e_y = dictionary[key][1]
            a = dictionary[key][2]
            e_x = e_x + row['S_x_err']
            e_y = e_x + row['S_y_err']
            a = a + 1
            dictionary[key] = [e_x, e_y, a]
            
    for key, value in dictionary.items():
        avg_error_x = value[0] / value[2]
        avg_error_y = value[1] / value[2]
        elev_depenent_stereographic_error = elev_depenent_stereographic_error.append(pd.DataFrame(
            data = [[key, avg_error_x, avg_error_y, value[2]]], 
            index = None, 
            columns = ['elevation_angle', 's_x_error', 's_y_error', 'No Images']))

    elev_depenent_stereographic_error = elev_depenent_stereographic_error.sort_values(by=['elevation_angle'])

    #cmse_real_df_error_azimuth.to_csv(evaluation_path + 'Net_568_Real_TD_Azimut_Fehler_in_Abhängigkeit_der_Elevation.csv', index = False)
    if(store_to_file):
        elev_depenent_stereographic_error.to_csv(eval_path + eval_file, index = False)
    return elev_depenent_stereographic_error
        


    
def elev_dependent_angular_real(dataframe, eval_path, eval_file, elevation_normalized = False, store_to_file = False):
    cmse_real_df_error_azimuth =  pd.DataFrame(data=None, index=None, columns=['elevation_angle', 'angular_error'])

    operation_df = dataframe

    dictionary = {}

    # Laufe über alle Einträge
    # Hashe Elevation_true
    # Falls Hash nicht im dict -> eintrag im dict mit {fehler, anzahl = 1}
    # Falls im dict -> fehler um aktuell erhöhen, anzahl++

    # für fertigen df
    # laufe über alle dict einträge
    # df der df-liste mit key (elevation) und fehler (fehler/anzahl) eintragen

    for index, row in operation_df.iterrows():
        true_elevation = row['Elevation_true'] * 45.0 + 45.0 if elevation_normalized else row['Elevation_true']
        key = round(true_elevation, 0)
        if key not in dictionary.keys():
            e = row['dot_angular_err_elevation']
            a = 1
            dictionary[key] = [e, a]

        else:
            e = dictionary[key][0]
            a = dictionary[key][1]
            e = e + row['dot_angular_err_elevation']
            a = a + 1
            dictionary[key] = [e, a]

    for key, value in dictionary.items():
        avg_error = value[0] / value[1]
        cmse_real_df_error_azimuth = cmse_real_df_error_azimuth.append(pd.DataFrame(
            data = [[key, avg_error, value[1]]], 
            index = None, 
            columns = ['elevation_angle', 'angular_error', 'No Images']))

    cmse_real_df_error_azimuth = cmse_real_df_error_azimuth.sort_values(by=['elevation_angle'])

    #cmse_real_df_error_azimuth.to_csv(evaluation_path + 'Net_568_Real_TD_Azimut_Fehler_in_Abhängigkeit_der_Elevation.csv', index = False)
    if(store_to_file):
        cmse_real_df_error_azimuth.to_csv(eval_path + eval_file, index = False)
    return cmse_real_df_error_azimuth




def elev_dependent_azimuth_real(dataframe, eval_path, eval_file, elevation_normalized = False, store_to_file = False):
    cmse_real_df_error_azimuth =  pd.DataFrame(data=None, index=None, columns=['elevation_angle', 'azimuth_error'])

    operation_df = dataframe

    dictionary = {}

    # Laufe über alle Einträge
    # Hashe Elevation_true
    # Falls Hash nicht im dict -> eintrag im dict mit {fehler, anzahl = 1}
    # Falls im dict -> fehler um aktuell erhöhen, anzahl++

    # für fertigen df
    # laufe über alle dict einträge
    # df der df-liste mit key (elevation) und fehler (fehler/anzahl) eintragen

    for index, row in operation_df.iterrows():
        true_elevation = row['Elevation_true'] * 45.0 + 45.0 if elevation_normalized else row['Elevation_true']
        key = round(true_elevation, 0)
        if key not in dictionary.keys():
            e = row['Azimuth_err']
            a = 1
            dictionary[key] = [e, a]

        else:
            e = dictionary[key][0]
            a = dictionary[key][1]
            e = e + row['Azimuth_err']
            a = a + 1
            dictionary[key] = [e, a]

    for key, value in dictionary.items():
        avg_error = value[0] / value[1]
        cmse_real_df_error_azimuth = cmse_real_df_error_azimuth.append(pd.DataFrame(
            data = [[key, avg_error, value[1]]], 
            index = None, 
            columns = ['elevation_angle', 'azimuth_error', 'No Images']))

    cmse_real_df_error_azimuth = cmse_real_df_error_azimuth.sort_values(by=['elevation_angle'])

    #cmse_real_df_error_azimuth.to_csv(evaluation_path + 'Net_568_Real_TD_Azimut_Fehler_in_Abhängigkeit_der_Elevation.csv', index = False)
    if(store_to_file):
        cmse_real_df_error_azimuth.to_csv(eval_path + eval_file, index = False)
    return cmse_real_df_error_azimuth

In [ ]:
fast_elev_dependent_angular_synthetic(net_df[0][0], eval_dir, '99_train-synth_test-synth.csv', store_to_file = True)
fast_elev_dependent_angular_synthetic(net_df[1][0], eval_dir, '204_train-synth_test-synth.csv', elevation_normalized = True, store_to_file = True)
fast_elev_dependent_angular_synthetic(net_df[2][0], eval_dir, '195_train-synth_test-synth.csv', store_to_file = True)

fast_elev_dependent_azimuth_synthetic(net_df[0][0], eval_dir, 'azimuth_99_train-synth_test-synth.csv', store_to_file = True)
fast_elev_dependent_azimuth_synthetic(net_df[1][0], eval_dir, 'azimuth_204_train-synth_test-synth.csv', elevation_normalized = True, store_to_file = True)
fast_elev_dependent_azimuth_synthetic(net_df[2][0], eval_dir, 'azimuth_195_train-synth_test-synth.csv', store_to_file = True)

elev_dependent_angular_real(net_df[0][1], eval_dir, '99_train-mixed_test-real.csv', store_to_file = True)
elev_dependent_angular_real(net_df[1][1], eval_dir, '204_train-mixed_test-real.csv', elevation_normalized = True, store_to_file = True)
elev_dependent_angular_real(net_df[2][1], eval_dir, '195_train-mixed_test-real.csv', store_to_file = True)

elev_dependent_azimuth_real(net_df[0][1], eval_dir, 'azimuth_99_train-mixed_test-real.csv', store_to_file = True)
elev_dependent_azimuth_real(net_df[1][1], eval_dir, 'azimuth_204_train-mixed_test-real.csv', elevation_normalized = True, store_to_file = True)
elev_dependent_azimuth_real(net_df[2][1], eval_dir, 'azimuth_195_train-mixed_test-real.csv', store_to_file = True)

fast_elev_dependent_stereographic_synthetic(net_df[2][0], eval_dir, 'stereographic_195_train-synth_test-synth.csv', store_to_file = True)
fast_elev_dependent_stereographic_synthetic(net_df[2][1], eval_dir, 'stereographic_195_train-mixed_test-real.csv', store_to_file = True)

In [ ]:
def do_Plot(title, eval_file, save = True):
    #fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (30,10))
    fig, ax = plt.subplots(figsize = (10,5))
    
    syn1 = fast_elev_dependent_angular_synthetic(net_df[0][0], None, None) # Net 99 Tr syn Te syn
    syn1.set_index('elevation_angle')
    syn2 = fast_elev_dependent_angular_synthetic(net_df[1][0], None, None, elevation_normalized = True) # Net 204 Tr syn Te syn
    syn2.set_index('elevation_angle')
    syn3 = fast_elev_dependent_angular_synthetic(net_df[2][0], None, None) # Net 195 Tr syn Te syn
    syn3.set_index('elevation_angle')
    
    syn_azi_1 = fast_elev_dependent_azimuth_synthetic(net_df[0][0], None, None) # Net 99 Tr syn Te syn
    syn_azi_1.set_index('elevation_angle')
    syn_azi_2 = fast_elev_dependent_azimuth_synthetic(net_df[1][0], None, None, elevation_normalized = True) # Net 204 Tr syn Te syn
    syn_azi_2.set_index('elevation_angle')
    syn_azi_3 = fast_elev_dependent_azimuth_synthetic(net_df[2][0], None, None) # Net 195 Tr syn Te syn
    syn_azi_3.set_index('elevation_angle')

    re_azi_1 = elev_dependent_azimuth_real(net_df[0][1], None, None) # Net 99 Tr mix Te real
    re_azi_1.set_index('elevation_angle')
    re_azi_2 = elev_dependent_azimuth_real(net_df[1][1], None, None, elevation_normalized = True) # Net 204 Tr mix Te real
    re_azi_2.set_index('elevation_angle')
    re_azi_3 = elev_dependent_azimuth_real(net_df[2][1], None, None) # Net 195 Tr mix Te real
    re_azi_3.set_index('elevation_angle')
    
    stereo_data = fast_elev_dependent_stereographic_synthetic(net_df[2][0], None, None)

    
    syn_dfs = [syn1[['elevation_angle', 'angular_error']], syn2['angular_error'], syn3['angular_error']]
    syn_azi_dfs = [syn_azi_1[['elevation_angle', 'azimuth_error']], syn_azi_2['azimuth_error']]
    re_azi_dfs = [re_azi_1[['elevation_angle', 'azimuth_error']], re_azi_2['azimuth_error']]
    stereo_dfs = [stereo_data[['elevation_angle', 's_x_error', 's_y_error']]]
    
    synth_plot = pd.concat(syn_dfs, join = 'outer', axis = 1)
    synth_plot.reset_index(drop = True, inplace = True)
    
    syn_azi_plot = pd.concat(syn_azi_dfs, join = 'outer', axis = 1)
    syn_azi_plot.reset_index(drop = True, inplace = True)
    
    re_azi_plot = pd.concat(re_azi_dfs, join = 'outer', axis = 1)
    re_azi_plot.reset_index(drop = True, inplace = True)
    
    stereo_plot = pd.concat(stereo_dfs, join = 'outer', axis = 1)
    stereo_plot.reset_index(drop = True, inplace = True)
    
    cmap = cm.get_cmap('tab20c')
    #colours = [cmap(0.), cmap(0.05), cmap(0.1), cmap(0.15), cmap(0.2), cmap(0.25), cmap(0.3), cmap(0.35)]
    colours = [cmap(0.4), cmap(0.45)]
        
    #synth_plot.plot(x = 'elevation_angle', kind = 'bar', ax = ax)
    #syn_azi_plot.plot(x = 'elevation_angle', kind = 'bar', ax = ax)
    stereo_plot.plot(x = 'elevation_angle', kind = 'bar', ax = ax, color = ['green', 'lightgreen'])
    #ax[1][1].axis('off')
    
    #fig.subplots_adjust(hspace = 0.3)
    
    ax.grid(True, ls = '--', axis = 'y')
    #ax[0][1].grid(True, ls = '--', axis = 'y')
    #ax[1][0].grid(True, ls = '--', axis = 'y')
    #ax[1][1].grid(True, ls = '--', axis = 'y')
    
    ax.set_ylabel('Mean Angular Error')
    ax.set_title('Elevation-dependent angular error on synth. test data using synth. training data')
    #ax[0][1].set_ylabel('Mean Azimuth Error')
    #ax[0][1].set_title('Elevation-dependent azimuth error on synth. test data using synth. training data')
    #ax[1][0].set_ylabel('Mean Stereographic Error')
    #ax[1][0].set_title('Elevation-dependent stereographic error sx, sy on synth. test data using synth. training data')
    #ax[1][1].set_ylabel('Mean Azimuth Error')
    #ax[1][1].set_title('Elevation-dependent azimuth error on real test data using mixed training data')
    
    ax.set_xlabel('Elevation Angle')
    #ax[0][1].set_xlabel('Elevation Angle')
    #ax[1][0].set_xlabel('Elevation Angle')
    #ax[1][1].set_xlabel('Elevation Angle')
    
    #ax.legend(['Net\_p,t', 'Net\_|p,t|', 'Net\_sx,sy'])
    ax.legend(['mean s\_x error', 'mean s\_y error'])
    
    for o in fig.findobj(text.Text):
        o.set_fontstyle('italic')
    
    if(save):
        tikzplotlib.save('{}.tex'.format(eval_file))
        plt.savefig('{}.png'.format(eval_file), format = 'png', bbox_inches = "tight", dpi = 300)

    plt.show()
    return

### Show Plot

In [ ]:
if Enable_Plotting:
    tsts_plot = do_Plot('Elevation-Dependent Mean Angular Error', eval_dir + 'Ele-Dep-Angular')